In [1]:
!pip install transformers datasets peft accelerate bitsandbytes


  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.4 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/10.4 MB 4.5 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/10.4 MB 4.2 MB/s eta 0:00:03
   ---------- ----------------------------- 2.6/10.4 MB 3.6 MB/s eta 0:00:03
   -------------- ------------------------- 3.7/10.4 MB 3.8 MB/s eta 0:00:02
   ------------------ --------------------- 4.7/10.4 MB 4.0 MB/s eta 0:00:02
   ---------------------- ----------------- 5.8/10.4 MB 4.2 MB/s eta 0:00:02
   ------------------------- -------------- 6.6/10.4 MB 4.2 MB/s eta 0:00:01
   ---------------------------- ----------- 7.3/10.4 MB 4.1 MB/s eta 0:00:01
   -------------------------------- ------- 8.4/10.4 MB 4.2 MB/s eta 0:00:01
   --------------


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import torch


c:\Users\abhis\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

encoded = dataset.map(tokenize_fn, batched=True)
encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])


c:\Users\abhis\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhis\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 425832.36 examples/s]
c:\Users\abh

In [4]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],  # for DistilBERT
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [7]:
training_args = TrainingArguments(
    output_dir="./lora-distilbert",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded["train"].shuffle(seed=42).select(range(2000)),  # Use small subset
    eval_dataset=encoded["test"].select(range(1000)),
)


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [8]:
trainer.train()


Step,Training Loss
50,0.689300
100,0.665200
150,0.629800
200,0.539300
250,0.392000
300,0.359400
350,0.424500
400,0.309700
450,0.346100
500,0.363100


TrainOutput(global_step=500, training_loss=0.4718281230926514, metrics={'train_runtime': 1797.8254, 'train_samples_per_second': 2.225, 'train_steps_per_second': 0.278, 'total_flos': 269478813696000.0, 'train_loss': 0.4718281230926514, 'epoch': 2.0})

In [9]:
model.save_pretrained("./lora-distilbert-sentiment")
tokenizer.save_pretrained("./lora-distilbert-sentiment")


('./lora-distilbert-sentiment\\tokenizer_config.json',
 './lora-distilbert-sentiment\\special_tokens_map.json',
 './lora-distilbert-sentiment\\vocab.txt',
 './lora-distilbert-sentiment\\added_tokens.json',
 './lora-distilbert-sentiment\\tokenizer.json')

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

# Step 1: Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./lora-distilbert-sentiment")

# Step 2: Load base model and apply LoRA adapters
base_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
model = PeftModel.from_pretrained(base_model, "./lora-distilbert-sentiment")

# Step 3: Define a prediction function
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(dim=1).item()
    return predicted_class

# Step 4: Test it on new sentences!
test_sentences = [
    "I love this movie, it was amazing!",
    "I hated the food, it was terrible.",
    "The product quality is fantastic!",
    "Worst service ever, I'm very disappointed.",
    "I,m dead"
]

for sentence in test_sentences:
    prediction = predict(sentence)
    print(f"Text: {sentence}")
    print(f"Predicted Class: {prediction}")
    print("-" * 50)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Text: I love this movie, it was amazing!
Predicted Class: 1
--------------------------------------------------
Text: I hated the food, it was terrible.
Predicted Class: 0
--------------------------------------------------
Text: The product quality is fantastic!
Predicted Class: 1
--------------------------------------------------
Text: Worst service ever, I'm very disappointed.
Predicted Class: 0
--------------------------------------------------
Text: I,m dead
Predicted Class: 0
--------------------------------------------------
